In [72]:
import numpy as np
import copy
import itertools

In [73]:
class Proportionality:
    def __init__(self, type_p, id1, id2):
        '''
        type_p = [-, +]
        id1 has P[type_p] on id2
        '''
        self.type = type_p
        self.id1 = id1
        self.id2 = id2
        
class Influence:
    def __init__(self, type_i, id1, id2):
        '''
        type_i = [-, +]
        id1 has I[type_p] on id2
        '''
        self.type = type_i
        self.id1 = id1
        self.id2 = id2
        
        

In [74]:
class variable:
    def __init__(self, id_state, list_proportionality, list_influence, possible_magnitude, possible_derivative, impossible_states, name):
        '''
        impossible_states - list of lists with impossible combinatiions of magnitude and derivative
        '''
        self.id = id_state
        self.proportionality = list_proportionality
        self.influence = list_influence
        self.possible_magnitude = possible_magnitude
        self.possible_derivative = possible_derivative
        self.impossible_states = impossible_states
        self.name = name
        self.magnitude = 'not assigned'
        self.derivative = 'not assigned'
    
    def print_state(self):
        print(self.name, ' M: ', self.magnitude,' D: ', self.derivative)
        
    
    def check_magnitude(self):
        if (self.magnitude == 2 and self.derivative == 1) or (self.magnitude == 0 and self.derivative == -1):
            return False
        else: 
            return True
            
            
            
    def assign_value(self, magnitude, derivative):
        if magnitude in self.possible_magnitude and derivative in self.possible_derivative and [magnitude, derivative] not in self.impossible_states:
            self.magnitude = magnitude
            self.derivative = derivative
            return True
        else:
            #print('Error')
            return False
            
            
class system:
    def __init__(self, v):
        '''
        s - list of states
        '''
        #self.id = id1
        self.vc = []
        self.variables = {}
        
        self.connection_to = set()
        self.connection_from = set()
        for k in v.keys():
            self.variables[k] = copy.deepcopy(v[k])

    
    def add_variable(self, v):
        self.variables[v.id] = copy.deepcopy(v)
        
    def add_id(self, id1):
        self.id = id1

    
    def print_system(self):
        print('SYSTEM: ')
        for s in self.variables:
            self.variables[s].print_state()
            
    def full_print(self):
        print('SYSTEM: ')
        for s in self.variables:
            self.variables[s].print_state()
            
        print('From ', self.id, ' such transitions are possible: ', self.connection_to)
        
        print('To ', self.id, ' are possible transitions from : ', self.connection_from)
        
            
    def add_vc(self, id1, id2, type_vc, value):
        '''
        dictionary with id1, id2, and name of the value which should be equal and the exact value 
        {'id1':id1, 'id2':id2, type_vc = 'magnitude', value = 2}
        '''
        vc_constr = {'id1':id1, 'id2':id2, 'type_vc' :type_vc, 'value': value}
        
        self.vc.append(vc_constr)
        
    def add_connection_to(self, c_t):
        self.connection_to.add(c_t)
    
    def add_connection_from(self, f_t):
        self.connection_from.add(f_t)
        
    def check_vc(self):
        if self.vc == []:
            return True
        
        for constr in self.vc:
            id1 = constr['id1']
            id2 = constr['id2']
            value = constr['value']
            if constr['type_vc'] == 'magnitude':
                if (self.variables[id1].magnitude == value) != (self.variables[id2].magnitude == value):
                    return False
                
            if constr['type_vc'] == 'derivative':
                if (self.variables[id1].derivative == value) != (self.variables[id2].derivative == value):
                    return False
        
        return True

                
    def check_system(self):
        
        for var in self.variables.values():
            if not var.check_magnitude():
                return False
            if [var.magnitude, var.derivative] in var.impossible_states:
                return False

            
        if not(self.check_vc()):
            return False
        
        possible = 0
        
        for var in self.variables.values():
            
            
            ID = var.id
            DER = var.derivative
            
            p_plus = [p.id1 for p in list_proportionality if p.type == '+' and p.id2 == ID]
            p_minus = [p.id1 for p in list_proportionality if p.type == '-' and p.id2 == ID]

            l_plus = [l.id1 for l in list_influence if l.type == '+' and l.id2 == ID]
            l_minus = [l.id1 for l in list_influence if l.type == '-' and l.id2 == ID]
            
            if len(p_plus) == 0 and len(p_minus) == 0 and len(l_minus) == 0 and len(l_plus) == 0:
                possible += 1
                continue
            
            
            possible_states = set()
            
            for id1 in p_plus:
                d = self.variables[id1].derivative
                if d == 1:
                    possible_states.add(1)
                    possible_states.add(0)
                    #possible_states.add(-1)
                #elif d == -1:
                #    possible_states.add(-1)
                #    possible_states.add(0)
                

            for id1 in p_minus:
                d = self.variables[id1].derivative
                if d == 1:
                    possible_states.add(-1)
                    possible_states.add(0)
                    #possible_states.add(1)
                #elif d == -1:
                #    possible_states.add(1)
                #    possible_states.add(0)
                    
            for id1 in l_plus:
                m = self.variables[id1].magnitude
                if m == 1:
                    possible_states.add(1)
                    possible_states.add(0)
                #if m == -1:
                #    possible_states.add(-1)

            for id1 in l_minus:
                m = self.variables[id1].magnitude
                if m == 1:
                    possible_states.add(-1)
                    possible_states.add(0)
                #if m == -1:
                #    possible_states.add(1)
                    
            if len(possible_states) == 0:
                possible_states.add(0)

            if 1 in possible_states and -1 in possible_states:
                possible_states.add(0)
   
            if DER not in possible_states:
                return False
        
            if DER in possible_states:
                possible += 1
        
        if possible == len(self.variables):
            return True
        
        return False
    
    


In [75]:
def check_continuity(s1, s2):
        '''
        from s1 to s2 
        '''
        for var in s1.variables.keys():
            if s1.variables[var].magnitude > 1 and s2.variables[var].magnitude < 1:
                return False
            
            if s1.variables[var].magnitude < 1 and s2.variables[var].magnitude > 1:
                return False
            
            if s1.variables[var].derivative > 0 and s2.variables[var].derivative < 0:
                return False
            
            if s1.variables[var].derivative < 0 and s2.variables[var].derivative > 0:
                return False
            
        return True

    
def check_variable(s1, s2):
    '''
    to omit the cases when (0, 0)->(+, +)
    '''
    for i in s2.variables.keys():
        if s1.variables[i].derivative != s2.variables[i].derivative and (s2.variables[i].magnitude != s1.variables[i].magnitude):
            return False
    return True

'''
def check_variable(s1, s2):
    for i in s2.variables.keys():
        if s1.variables[i].derivative == 0 and (s2.variables[i].magnitude != s1.variables[i].magnitude):
            return False
    return True
'''

def check_next_state(prev_state, new_state, print_ = False):
    error = False
    

    
    if not(check_variable(prev_state, new_state)):
        #print('hi')
        return False
    
    for var in new_state.variables.values():
        ID = var.id
        DER = var.derivative
        p_plus = [(p.id1, p.id2) for p in list_proportionality if p.type == '+' and p.id2 == ID]
        p_minus = [(p.id1, p.id2) for p in list_proportionality if p.type == '-' and p.id2 == ID]

        l_plus = [(l.id1, l.id2) for l in list_influence if l.type == '+' and l.id2 == ID]
        l_minus = [(l.id1, l.id2) for l in list_influence if l.type == '-' and l.id2 == ID]


        if len(p_plus) == 0 and len(p_minus) == 0 and len(l_minus) == 0 and len(l_plus) == 0:
            if prev_state.variables[ID].derivative > 0:
                if new_state.variables[ID].magnitude < prev_state.variables[ID].magnitude:
                    error = True
                    
            if prev_state.variables[ID].derivative < 0 :
                if new_state.variables[ID].magnitude > prev_state.variables[ID].magnitude:
                    error = True
        
        possible_states = set()

        for (id1, id2) in p_plus:
            
            d = new_state.variables[id1].derivative
            if d == 1:
                if prev_state.variables[id2].derivative == 0 and new_state.variables[id2].derivative != 1:
                    
                    #print('pp')
                    error = True
                elif prev_state.variables[id2].derivative == -1 and new_state.variables[id2].derivative != 0:
                    #print('pp')
                    error = True

        for (id1, id2) in p_minus:
            #print('pm')
            d = new_state.variables[id1].derivative
            if d == 1:
                if prev_state.variables[id2].derivative == 0 and new_state.variables[id2].derivative != -1:
                    error = True
                elif prev_state.variables[id2].derivative == 1 and new_state.variables[id2].derivative != 0:
                    error = True

        for (id1, id2) in l_plus:
            #print('lp')
            m = new_state.variables[id1].magnitude
            if m == 1:
               
                if prev_state.variables[id2].derivative == 0 and new_state.variables[id1].derivative != 1:
                    #print('lp')
                    error = True
                elif prev_state.variables[id2].derivative == -1 and new_state.variables[id1].derivative != 0:
                    #print('lp')
                    error = True

        for (id1, id2) in l_minus:
            
            m = new_state.variables[id1].magnitude
            if m == 1:
                if prev_state.variables[id2].derivative == 0 and new_state.variables[id2].derivative != -1:
                    #print('lm')
                    error = True
                elif prev_state.variables[id2].derivative == 1 and new_state.variables[id2].derivative != 0:
                    #print('lm')
                    error = True
    if error:
        if print_:
            print('State transition impossible!')
        return False
    else:
        if print_:
            print('State transition OK')
        return True

In [76]:
P1 = Proportionality('+', 5, 3)
P2 = Proportionality('+', 2, 4)
P3 = Proportionality('+', 4, 5)

I1 = Influence('-', 3, 2)
I2 = Influence('+', 1, 2)

list_influence = [I1, I2]
list_proportionality = [P1, P2, P3]

In [77]:
Inflow = variable(1, list_proportionality, list_influence, [0, 1], [-1, 0, 1], [0, -1], 'Inflow')
Outflow = variable(3, list_proportionality, list_influence, [0, 1, 2], [-1, 0, 1], [], 'Outflow')
Volume = variable(2, list_proportionality, list_influence, [0, 1, 2], [-1, 0, 1], [], 'Volume')
Height = variable(4, list_proportionality, list_influence, [0, 1, 2], [-1, 0, 1], [0, -1], 'Height')
Pressure = variable(5, list_proportionality, list_influence, [0, 1, 2], [-1, 0, 1], [0, -1], 'Pressure')

System = system({Inflow.id:Inflow, Volume.id:Volume, Outflow.id:Outflow, Height.id:Height, Pressure.id:Pressure})
System.add_vc(2, 3, 'magnitude', 2)
System.add_vc(2, 4, 'magnitude', 2)
System.add_vc(2, 5, 'magnitude', 2)
System.add_vc(2, 3, 'magnitude', 0)
System.add_vc(2, 4, 'magnitude', 0)
System.add_vc(2, 5, 'magnitude', 0)

derivatives = []
magnitudes = []

possible_states = []
states = list(System.variables.values())

for s in states:
    
    
    derivatives.append(s.possible_derivative)
    magnitudes.append(s.possible_magnitude)
    p_s = []
    for d in s.possible_derivative:
        for m in s.possible_magnitude:
            r = copy.deepcopy(s)
            r.assign_value(m, d)
            p_s.append(r)
            
        
    possible_states.append(p_s)
            
all_possible_states_of_system = []
for element in itertools.product(*possible_states):
    #print(len(element))
    all_possible_states_of_system.append(system({var.id:var for var in element}))
    #all_possible_states_of_system.append(system((list(element))))
        

In [78]:
for i in range(len(all_possible_states_of_system)):
    all_possible_states_of_system[i].add_vc(2, 3, 'magnitude', 0)
    all_possible_states_of_system[i].add_vc(2, 3, 'magnitude', 2)
    all_possible_states_of_system[i].add_vc(2, 4, 'magnitude', 0)
    all_possible_states_of_system[i].add_vc(2, 4, 'magnitude', 2)
    all_possible_states_of_system[i].add_vc(2, 5, 'magnitude', 0)
    all_possible_states_of_system[i].add_vc(2, 5, 'magnitude', 2)

In [79]:
j = 0
possible_indices = []
for i in range(len(all_possible_states_of_system)):
    #print(all_possible_states_of_system[i].check_system())
    #print(all_possible_states_of_system[i].check_system())
    if all_possible_states_of_system[i].check_system() == True:
        j+=1
        #print('hi')
        print(i)
        possible_indices.append(i)
        all_possible_states_of_system[i].print_system()
        all_possible_states_of_system[i].add_id(i)
        #print(j)
j

7654
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  1  D:  -1
Outflow  M:  1  D:  0
Height  M:  1  D:  0
Pressure  M:  1  D:  0
9021
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  0
Outflow  M:  0  D:  0
Height  M:  0  D:  0
Pressure  M:  0  D:  0
9841
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  1  D:  0
Outflow  M:  1  D:  0
Height  M:  1  D:  0
Pressure  M:  1  D:  0
10661
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  2  D:  0
Outflow  M:  2  D:  0
Height  M:  2  D:  0
Pressure  M:  2  D:  0
11208
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  1
Outflow  M:  0  D:  0
Height  M:  0  D:  0
Pressure  M:  0  D:  0
11235
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  1
Outflow  M:  0  D:  0
Height  M:  0  D:  1
Pressure  M:  0  D:  0
11238
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  1
Outflow  M:  0  D:  0
Height  M:  0  D:  1
Pressure  M:  0  D:  1
11481
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  1
Outflow  M:  0  D:  1
Height  M:  0  D:  1
Pressure  M:  0  D:  1
12028
SYST

44

In [80]:
list_of_states = []
for i in possible_indices:

    k = 0
    for j in possible_indices:

        if check_continuity(all_possible_states_of_system[i], all_possible_states_of_system[j]) and check_next_state(all_possible_states_of_system[i], all_possible_states_of_system[j], print_ = False):
            flag1 = 1
            all_possible_states_of_system[i].add_connection_to(j)
            
        if check_continuity(all_possible_states_of_system[j], all_possible_states_of_system[i]) and check_next_state(all_possible_states_of_system[j], all_possible_states_of_system[i], print_ = False):
            flag2 = 1
            all_possible_states_of_system[i].add_connection_from(j)
        
    print(len(all_possible_states_of_system[i].connection_to))
    print(len(all_possible_states_of_system[i].connection_from))
    
    if len(all_possible_states_of_system[i].connection_to) == 0:

        print(i, '  NO TO')

        
        
    if len(all_possible_states_of_system[i].connection_from) == 0:
        print(i, '  NO FROM')
        k = 1

        
    if k != 1:
        list_of_states.append(i)

2
0
7654   NO FROM
0
8
9021   NO TO
0
8
9841   NO TO
0
0
10661   NO TO
10661   NO FROM
4
6
6
4
8
2
10
10
2
0
12028   NO FROM
2
0
12055   NO FROM
2
0
12058   NO FROM
4
0
12301   NO FROM
6
6
2
10
6
7
2
6
4
4
3
12
6
16
2
0
23783   NO FROM
7
9
10
6
13
3
16
15
4
0
25150   NO FROM
4
0
25177   NO FROM
4
0
25180   NO FROM
7
0
25423   NO FROM
4
4
4
4
9
2
3
4
2
3
2
14
3
8
1
6
4
6
6
4
8
2
10
13
2
0
38272   NO FROM
2
0
38299   NO FROM
2
0
38302   NO FROM
4
0
38545   NO FROM


In [81]:
for  i in list_of_states:
    all_possible_states_of_system[i].full_print()

SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  0
Outflow  M:  0  D:  0
Height  M:  0  D:  0
Pressure  M:  0  D:  0
From  9021  such transitions are possible:  set()
To  9021  are possible transitions from :  {11235, 24357, 11238, 11208, 24360, 24330, 11481, 24603}
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  1  D:  0
Outflow  M:  1  D:  0
Height  M:  1  D:  0
Pressure  M:  1  D:  0
From  9841  such transitions are possible:  set()
To  9841  are possible transitions from :  {12301, 25423, 12055, 25177, 12058, 12028, 25150, 25180}
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  1
Outflow  M:  0  D:  0
Height  M:  0  D:  0
Pressure  M:  0  D:  0
From  11208  such transitions are possible:  {11481, 24603, 9021, 22143}
To  11208  are possible transitions from :  {11235, 24357, 11238, 24360, 11481, 24603}
SYSTEM: 
Inflow  M:  1  D:  -1
Volume  M:  0  D:  1
Outflow  M:  0  D:  0
Height  M:  0  D:  1
Pressure  M:  0  D:  0
From  11235  such transitions are possible:  {11208, 24330, 11481, 2

In [82]:
len(list_of_states)

29

In [83]:
to_remove = []
for j in list_of_states:
    flag = 0
    for node in all_possible_states_of_system[j].connection_from:
        if node in list_of_states:
            flag = 1
    if flag == 0 and len(all_possible_states_of_system[j].connection_to) == 0:
        #print(i)
        to_remove.append(j)

        
list_of_states_new = []
for j in list_of_states:
    if j not in to_remove:
        list_of_states_new.append(j)

In [84]:
len(list_of_states_new)

28

In [85]:
def get_valid_states(all_states, list_of_states):
    valid_states = copy.deepcopy(np.array(all_states)[list_of_states])
    for i in range(len(valid_states)):
        to = list(valid_states[i].connection_to)
        from_ = list(valid_states[i].connection_from)
        for j in to:
            if j not in list_of_states:
                valid_states[i].connection_to.remove(j)
        for k in from_:
            if k not in list_of_states:
                valid_states[i].connection_from.remove(k)

    trans = {}
    for i in range(len(list_of_states)):
        trans[list_of_states[i]] = i

    for i in range(len(valid_states)):
        to = list(valid_states[i].connection_to)
        from_ = list(valid_states[i].connection_from)
        new_to = []
        new_from = []
        for j in to:
            try:
                new_to.append(trans[j])
            except KeyError:
                new_to.append(j)
        for k in from_:
            try:
                new_from.append(trans[k])
            except KeyError:
                new_from.append(k)
        valid_states[i].connection_to = set(new_to)
        valid_states[i].connection_from = set(new_from)
    return valid_states

valid_states = get_valid_states(all_possible_states_of_system, list_of_states_new)

In [86]:
#list_of_states_new = list(range(len(valid_states)))

In [87]:
#list_of_states_new

In [88]:
import pygraphviz as pgv

graph = pgv.AGraph(directed=True, fixedsize=True)

def graph_state(state):
    
    def tm(val): #translate magnitude
        if val == 1:
            return '+'
        elif val == 2:
            return 'max'
        else:
            return val
    def td(val): #translate derivative
        if val == 1:
            return '+'
        elif val == -1:
            return '-'
        else:
            return val
        
    return 'I(%s, %s)\nV(%s, %s)\nO(%s, %s)\nH(%s, %s)\nP(%s, %s)' % (tm(state.variables[1].magnitude), \
                                               td(state.variables[1].derivative), \
                                               tm(state.variables[2].magnitude), \
                                               td(state.variables[2].derivative), \
                                               tm(state.variables[3].magnitude), \
                                               td(state.variables[3].derivative),\
                                               tm(state.variables[4].magnitude), \
                                               td(state.variables[4].derivative),
                                               tm(state.variables[5].magnitude), \
                                               td(state.variables[5].derivative)) 

for i in range(len(valid_states)):
    graph.add_node(graph_state(valid_states[i]), color='red', style = 'filled, bold')
num_edges = 0
for i in range(len(valid_states)):
    for j in valid_states[i].connection_from:
        num_edges+=1
        graph.add_edge(graph_state(valid_states[j]), graph_state(valid_states[i]), timestep = True)

print('#edges ', num_edges)        
graph.layout(prog='dot')

graph.draw('graph1_extended.png')

#edges  159


In [89]:
valid_states.shape

(28,)